Import Packages & Libraries

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime

Set URL of album to parse and User Agent info of browser to spoof

In [5]:
#url = 'http://pitchfork.com/reviews/albums/21864-strangers/'
#url = 'http://pitchfork.com/reviews/albums/'
#url = 'http://pitchfork.com/reviews/albums/21673-teens-of-denial/'
#url = 'http://pitchfork.com/reviews/albums/21654-it-came-from-nyc/'
#url = 'http://pitchfork.com/reviews/albums/21980-fantasma/'
#url = 'http://pitchfork.com/reviews/albums/15996-lou-reed-metallica/'
url = 'http://pitchfork.com/reviews/albums/21997-blonde-on-blonde-revisited/'
ua ={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}

Get web page & print reponse code - 200 means the page was retrieved successfully

In [6]:
r = requests.get(url, headers=ua)
retrievalTime = datetime.now()
print(type(r.status_code))

<class 'int'>


Create BeautifulSoup object for the particular review

In [7]:
reviewSoup = BeautifulSoup(r.text, "html.parser")
#print(reviewSoup.prettify())
tombs = reviewSoup.findAll(class_='tombstone')
reviewTitle = tombs[0].find(class_='review-title')
print(reviewTitle)

<h1 class="review-title" data-reactid=".2bhgugx4dts.1.6:0.1.0.$21997/=1$21997.0.1.0.0.1">Blonde on Blonde Revisited</h1>


Get BeautifulSoup Elements of Review Components

In [8]:
reviewIDElement = reviewSoup.find(attrs={"rel": "canonical"})
artistElement = reviewSoup.find(class_='artists')
albumsElement = reviewSoup.find(class_='review-title')
albumArtElement = reviewSoup.find(class_='album-art')
labelsElement = reviewSoup.find(class_='labels-and-years')
yearElement = reviewSoup.find(class_='year')
bnmElement = reviewSoup.find(class_='bnm-txt')
scoreElement = reviewSoup.find(class_='score')
writerElement = reviewSoup.find(class_='display-name')
genreElement = reviewSoup.find(class_='genre-list')
publishDateElement = reviewSoup.find(class_='pub-date')
reviewContent = reviewSoup.find(class_='contents dropcap')

Review ID & Link

In [9]:
reviewLink = reviewIDElement['href']
print(reviewLink)
reviewID = int(reviewLink[36:reviewLink.find('-')])
print(reviewID)

http://pitchfork.com/reviews/albums/21997-blonde-on-blonde-revisited/
21997


Artist Name & ID

In [10]:
try:
    artist = artistElement.find("li")
    #print(artist)
    artistHref = artist.find("a")['href']
    artistName = artist.text
    artistID = int(artistHref[9:artistHref.find('-')])
except:
    artistName = artistElement.text
    artistID = 0
    
print(artistName)
print(artistID)

Various Artists
0


Album Name

In [11]:
albumName = str(albumsElement.contents[0])
print(albumName)

Blonde on Blonde Revisited


Album Art Link

In [12]:
albumArtLink = albumArtElement.contents[0]['src']
print(albumArtLink)

http://cdn4.pitchfork.com/albums/23342/homepage_large.bb2fb299.jpg


Record Labels

In [13]:
recordLabels = labelsElement.find_all('li')
labelList = [l.contents[0] for l in recordLabels]
print(labelList)

['Mojo']


Release Year & Reissue flag

In [24]:
yearText = yearElement.contents[1].text
if '/' in yearText:
    isReissue = 1
    releaseYear = yearText[0:yearText.index('/')]
else:
    isReissue = 0
    releaseYear = yearText
    
print(type(isReissue))
print(type(releaseYear))

<class 'int'>
<class 'str'>


Score & BNM/BNR tag

In [15]:
score = float(scoreElement.contents[0])
print(score)
try:
    isBNR = 1 if 'reissue' in bnmElement.text else 0
    isBNM = 1 if 'music' in bnmElement.text else 0
except:
    isBNM = 0
    isBNR = 0
    
print(isBNM)
print(isBNR)

7.6
0
0


Writer

In [16]:
writer = str(writerElement.contents[0])
print(writer)

Sam Sodomsky


Genres

In [17]:
genreLis = genreElement.find_all('li')
genreList = [g.find('a').contents[0] for g in genreLis]
print(genreList)

[]


Publish Date

In [18]:
timeStr = publishDateElement.contents[0]
publishDate = datetime.strptime(timeStr, '%b %d %Y')
print(type(publishDate))
print(publishDate)

ValueError: time data 'June 8 2016' does not match format '%b %d %Y'

Raw Review Text

In [19]:
reviewContent = str(reviewContent)
print(type(reviewContent))
print(reviewContent)

<class 'str'>
<div class="contents dropcap" data-reactid=".2bhgugx4dts.1.6:0.1.0.$21997/=1$21997.0.3.2.0.1.0.1"><p><span style="font-weight: 400;">One of the most famous Dylanisms arose from a conversation with director Sam Peckinpah on the set of the 1973 movie </span><i><span style="font-weight: 400;">Pat Garrett and Billy the Kid</span></i><span style="font-weight: 400;">. “Just be yourself,” Peckinpah suggested to a 32-year-old <a href="http://pitchfork.com/artists/1177-bob-dylan/">Bob Dylan</a> who, by this point, had already been everything from folk messiah to rock Judas, who had received glowing reviews and miserable ones, who once stood as a pillar of New York authenticity and was now struggling to play the role of a mysterious stranger in a Western movie filmed in Durango, Mexico. Fittingly, Dylan snapped back to Peckinpah: “Which one?</span><span style="font-weight: 400;">”</span></p>
<p><span style="font-weight: 400;"> If Dylan’s 1966 double album </span><i><span style="fon

Print relevant info

In [23]:
print(type(artistID))
print(type(artistName))
print(type(reviewID))
print(type(albumName))
print(type(reviewLink))
print(type(albumArtLink))
print(type(genreList))
print(type(labelList))
print(type(writer))
print(type(score))
print(type(isBNM))
print(type(isBNR))
print(type(publishDate))
print(type(retrievalTime))
print(type(reviewContent))

<class 'int'>
<class 'str'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'list'>
<class 'list'>
<class 'str'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'datetime.datetime'>
<class 'str'>


In [26]:
print(artistID)
reviewDict = {'artistID':artistID}
print(artistName)
reviewDict['artistName'] = artistName
print(reviewID)
reviewDict['reviewID'] = reviewID
print(albumName)
reviewDict['albumName'] = albumName
print(reviewLink)
reviewDict['reviewLink'] = reviewLink
print(albumArtLink)
reviewDict['albumArtLink'] = albumArtLink
print(genreList)
reviewDict['genreList'] = genreList
print(labelList)
reviewDict['labelList'] = labelList
print(writer)
reviewDict['writer'] = writer
print(score)
reviewDict['score'] = score
print(isBNM)
reviewDict['isBNM'] = isBNM
print(isBNR)
reviewDict['isBNR'] = isBNR
print(publishDate)
reviewDict['publishDate'] = publishDate
print(retrievalTime)
reviewDict['retrievalTime'] = retrievalTime
print(reviewContent)
reviewDict['reviewContent'] = reviewContent

32785
Car Seat Headrest
21673
Teens of Denial
http://pitchfork.com/reviews/albums/21673-teens-of-denial/
http://cdn.pitchfork.com/albums/22996/homepage_large.2365f6ed.jpg
['Rock']
['Matador']
Jeremy Gordon
8.5
1
0
2016-05-20 00:00:00
2016-06-07 07:20:23.114909
<div class="contents dropcap" data-reactid=".m66t7y86bk.1.6:0.1.0.$21673/=1$21673.0.3.2.0.1.0.1"><p><span style="font-weight: 400;">Sometimes, drugs are no fun. The rad night you imagined, watching </span><i><span style="font-weight: 400;">2001: A Space Odyssey</span></i><span style="font-weight: 400;"> and brushing against the outer boundaries of your consciousness, becomes a six-hour hell of wondering </span><i><span style="font-weight: 400;">Did I leave the oven on? </span></i><span style="font-weight: 400;">or </span><i><span style="font-weight: 400;">Did I look weird when I said that thing to that one person</span></i><span style="font-weight: 400;"> or</span><i><span style="font-weight: 400;"> Do I just </span></i><span sty

In [29]:
multiDict = []
multiDict.append(reviewDict)

In [404]:
reviewDataFrame = pd.DataFrame([reviewDict])

In [407]:
print(reviewDataFrame)

                                        albumArtLink  albumName  artistID  \
0  http://cdn.pitchfork.com/albums/23204/homepage...  Strangers      3108   

       artistName       genreList  isBNM  isBNR       labelList publishDate  \
0  Marissa Nadler  [Folk/Country]      0      0  [Sacred Bones]  2016-05-21   

               retrievalTime  \
0 2016-06-04 14:16:39.627244   

                                       reviewContent  reviewID  \
0  <div class="contents dropcap" data-reactid=".1...     21864   

                                          reviewLink  score           writer  
0  http://pitchfork.com/reviews/albums/21864-stra...    7.6  Marcus J. Moore  


In [426]:
print(reviewDict)
reviewDataFrame.append([reviewDict])

{'artistID': 32785, 'isBNM': 1, 'reviewLink': 'http://pitchfork.com/reviews/albums/21673-teens-of-denial/', 'artistName': 'Car Seat Headrest', 'publishDate': datetime.datetime(2016, 5, 20, 0, 0), 'reviewContent': <div class="contents dropcap" data-reactid=".76eqj8ary8.1.6:0.1.0.$21673/=1$21673.0.3.2.0.1.0.1"><p><span style="font-weight: 400;">Sometimes, drugs are no fun. The rad night you imagined, watching </span><i><span style="font-weight: 400;">2001: A Space Odyssey</span></i><span style="font-weight: 400;"> and brushing against the outer boundaries of your consciousness, becomes a six-hour hell of wondering </span><i><span style="font-weight: 400;">Did I leave the oven on? </span></i><span style="font-weight: 400;">or </span><i><span style="font-weight: 400;">Did I look weird when I said that thing to that one person</span></i><span style="font-weight: 400;"> or</span><i><span style="font-weight: 400;"> Do I just </span></i><span style="font-weight: 400;">think</span><i><span styl

,albumArtLink,albumName,artistID,artistName,genreList,isBNM,isBNR,labelList,publishDate,retrievalTime,reviewContent,reviewID,reviewLink,score,writer
0,http://cdn.pitchfork.com/albums/23204/homepage...,Strangers,3108,Marissa Nadler,[Folk/Country],0,0,[Sacred Bones],2016-05-21,2016-06-04 14:16:39.627244,"<div class=""contents dropcap"" data-reactid="".1...",21864,http://pitchfork.com/reviews/albums/21864-stra...,7.6,Marcus J. Moore
0,http://cdn.pitchfork.com/albums/22996/homepage...,Teens of Denial,32785,Car Seat Headrest,[Rock],1,0,[Matador],2016-05-20,2016-06-04 16:42:51.568383,"<div class=""contents dropcap"" data-reactid="".7...",21673,http://pitchfork.com/reviews/albums/21673-teen...,8.5,Jeremy Gordon


In [429]:
indexFrame = reviewDataFrame.set_index('reviewID')

In [432]:
indexFrame.append([reviewDict])

,albumArtLink,albumName,artistID,artistName,genreList,isBNM,isBNR,labelList,publishDate,retrievalTime,reviewContent,reviewID,reviewLink,score,writer
reviewID,,,,,,,,,,,,,,,
21864,http://cdn.pitchfork.com/albums/23204/homepage...,Strangers,3108,Marissa Nadler,[Folk/Country],0,0,[Sacred Bones],2016-05-21,2016-06-04 14:16:39.627244,"<div class=""contents dropcap"" data-reactid="".1...",NaN,http://pitchfork.com/reviews/albums/21864-stra...,7.6,Marcus J. Moore
0,http://cdn.pitchfork.com/albums/22996/homepage...,Teens of Denial,32785,Car Seat Headrest,[Rock],1,0,[Matador],2016-05-20,2016-06-04 16:42:51.568383,"<div class=""contents dropcap"" data-reactid="".7...",21673.0,http://pitchfork.com/reviews/albums/21673-teen...,8.5,Jeremy Gordon


In [463]:
reviewDataFrame.set_index(reviewID)

KeyError: 21673

In [482]:
indexFrame = indexFrame.set_index(reviewID)

KeyError: 21673

In [492]:
indexyFrame

,albumArtLink,albumName,artistID,artistName,genreList,isBNM,isBNR,labelList,publishDate,retrievalTime,reviewContent,reviewLink,score,writer
reviewID,,,,,,,,,,,,,,
21864,http://cdn.pitchfork.com/albums/23204/homepage...,Strangers,3108,Marissa Nadler,[Folk/Country],0,0,[Sacred Bones],2016-05-21,2016-06-04 14:16:39.627244,"<div class=""contents dropcap"" data-reactid="".1...",http://pitchfork.com/reviews/albums/21864-stra...,7.6,Marcus J. Moore
21673,http://cdn.pitchfork.com/albums/22996/homepage...,Teens of Denial,32785,Car Seat Headrest,[Rock],1,0,[Matador],2016-05-20,2016-06-04 16:42:51.568383,"<div class=""contents dropcap"" data-reactid="".7...",http://pitchfork.com/reviews/albums/21673-teen...,8.5,Jeremy Gordon
